# RepText Arabic Training - Quick Start Guide

This notebook demonstrates how to pretrain RepText for Arabic text generation.

## Prerequisites
- NVIDIA GPU with at least 24GB VRAM
- Python 3.8+
- CUDA 11.7+

## Step 1: Install Dependencies

In [ ]:
!pip install -r requirements.txt

## Step 2: Download Arabic Fonts

In [ ]:
# Download recommended Arabic fonts from Google Fonts
!python download_arabic_fonts.py

In [ ]:
# Verify fonts were downloaded
import os
fonts = [f for f in os.listdir('arabic_fonts') if f.endswith(('.ttf', '.otf'))]
print(f"Found {len(fonts)} fonts:")
for font in fonts:
    print(f"  - {font}")

## Step 3: Test Font Rendering

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Test Arabic text rendering
test_text = "مرحبا بكم في RepText"
font_path = "./arabic_fonts/Amiri-Regular.ttf"

img = Image.new('RGB', (600, 150), color='white')
draw = ImageDraw.Draw(img)
font = ImageFont.truetype(font_path, 60)
draw.text((50, 40), test_text, font=font, fill='black')

plt.figure(figsize=(10, 3))
plt.imshow(img)
plt.axis('off')
plt.title('Arabic Text Rendering Test')
plt.show()

print("✓ Arabic text rendering works correctly!")

## Step 4: Prepare Training Dataset

This will generate synthetic training samples with:
- Glyph images (rendered Arabic text)
- Position maps (location heatmaps)
- Binary masks (text regions)
- Canny edges

In [ ]:
# For quick testing, use a small number of samples
# For actual training, use 10000-50000
NUM_SAMPLES = 100  # Change to 10000 for real training

!python prepare_arabic_dataset.py \
    --output_dir ./arabic_training_data \
    --font_dir ./arabic_fonts \
    --text_file ./arabic_texts.txt \
    --num_samples {NUM_SAMPLES} \
    --width 1024 \
    --height 1024 \
    --min_font_size 60 \
    --max_font_size 120

## Step 5: Visualize Training Samples

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

# Load a sample
sample_dir = "./arabic_training_data/sample_000000"

# Load images
glyph = Image.open(f"{sample_dir}/glyph.png")
position = Image.open(f"{sample_dir}/position.png")
mask = Image.open(f"{sample_dir}/mask.png")
canny = Image.open(f"{sample_dir}/canny.png")

# Load metadata
with open(f"{sample_dir}/metadata.json", 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(glyph)
axes[0, 0].set_title(f"Glyph: {metadata['text']}")
axes[0, 0].axis('off')

axes[0, 1].imshow(position)
axes[0, 1].set_title("Position Map")
axes[0, 1].axis('off')

axes[1, 0].imshow(mask, cmap='gray')
axes[1, 0].set_title("Text Mask")
axes[1, 0].axis('off')

axes[1, 1].imshow(canny)
axes[1, 1].set_title("Canny Edges")
axes[1, 1].axis('off')

plt.suptitle(f"Training Sample - Font Size: {metadata['font_size']}px")
plt.tight_layout()
plt.show()

## Step 6: Test Dataset Loading

In [ ]:
from arabic_dataset import create_dataloaders

# Create dataloaders
train_loader, val_loader = create_dataloaders(
    data_dir='./arabic_training_data',
    batch_size=2,
    num_workers=0,
    image_size=(1024, 1024),
    train_ratio=0.9
)

# Test loading a batch
batch = next(iter(train_loader))

print("Batch contents:")
print(f"  Glyph shape: {batch['glyph'].shape}")
print(f"  Position shape: {batch['position'].shape}")
print(f"  Mask shape: {batch['mask'].shape}")
print(f"  Canny shape: {batch['canny'].shape}")
print(f"  Text samples: {batch['text']}")
print(f"  Font sizes: {batch['font_size']}")
print("\n✓ Dataset loading works correctly!")

## Step 7: Configure Accelerate for Training

In [ ]:
!accelerate config

## Step 8: Review Training Configuration

In [ ]:
import yaml

with open('train_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Training Configuration:")
print(yaml.dump(config, default_flow_style=False))

## Step 9: Launch Training

**Note:** Training takes a long time. You may want to run this in a terminal instead of notebook.

For terminal:
```bash
accelerate launch train_arabic.py --config train_config.yaml
```

Or use the automated script:
```bash
./train_arabic.sh
```

In [ ]:
# For notebook training (not recommended for long training runs)
# Uncomment to run:
# !accelerate launch train_arabic.py --config train_config.yaml

print("It's recommended to run training in a terminal or tmux session:")
print("  accelerate launch train_arabic.py --config train_config.yaml")

## Step 10: Monitor Training (Optional - with W&B)

In [ ]:
# Install wandb
# !pip install wandb
# !wandb login

# Then run with --use_wandb flag:
# !accelerate launch train_arabic.py --config train_config.yaml --use_wandb

## Step 11: Test Your Trained Model

After training completes, test the model:

In [ ]:
import torch
from controlnet_flux import FluxControlNetModel
from pipeline_flux_controlnet import FluxControlNetPipeline
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

# Load your trained model
base_model = "black-forest-labs/FLUX.1-dev"
controlnet_model = "./output/arabic_reptext/final_model"  # or checkpoint-XXXX

print("Loading models...")
controlnet = FluxControlNetModel.from_pretrained(
    controlnet_model, 
    torch_dtype=torch.bfloat16
)
pipe = FluxControlNetPipeline.from_pretrained(
    base_model, 
    controlnet=controlnet, 
    torch_dtype=torch.bfloat16
).to("cuda")

print("✓ Models loaded successfully!")

In [ ]:
# Generate an image with Arabic text
# (Use the inference code from infer.py)

print("Ready to generate images with Arabic text!")
print("See infer.py for complete inference examples.")

## Summary

You've completed:
✅ Installed dependencies
✅ Downloaded Arabic fonts
✅ Prepared training dataset
✅ Tested dataset loading
✅ Configured training

Next steps:
1. Increase `NUM_SAMPLES` to 10000+ for real training
2. Run training in a terminal session
3. Monitor with W&B or TensorBoard
4. Test your trained model

For detailed documentation, see [TRAINING_GUIDE.md](TRAINING_GUIDE.md)